In [114]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from scipy.fftpack import fft

In [115]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [116]:
# Load the actual data from the Excel file
actual_data = pd.read_excel("../../DATAFORMODELtrain200824.xlsx")

# Convert 'Date' to datetime format and round to the nearest hour
actual_data['Date'] = pd.to_datetime(actual_data['Date'], format='%m/%d/%Y %H:%M').dt.round('h')

In [117]:
# Define the date range
start_date = pd.to_datetime('2017-01-01 00:00')
cutoff_date = pd.to_datetime("2024-08-20 23:00")

In [118]:
# Filter data within the date range and drop unnecessary columns
columns_to_drop = [
    'Y', 'M', 'Day', 'H', 'Y2016',	'Y2017',	'Y2018',	'Y2019',	'Y2020',	'Y2021',	'Y2022',	'Y2023',	'Y2024',
    'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10', 'M11', 'M12',
    'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'h8', 'h9', 'h10',
    'h11', 'h12', 'h13', 'h14', 'h15', 'h16', 'h17', 'h18', 'h19',
    'h20', 'h21', 'h22', 'h23', 'h24',
    'PriceCZ', 'PriceSK', 'PriceRO', 'WDAY'
]
data = actual_data[(actual_data['Date'] >= start_date) & (actual_data['Date'] <= cutoff_date)].drop(columns=columns_to_drop)


In [119]:
def check_NaN(data):
    # Select only numeric columns for spline interpolation
    numeric_columns = data.select_dtypes(include=[np.number]).columns

    print("Number of NaN values:\n", data[numeric_columns].isna().sum())

In [120]:
check_NaN(data)

Number of NaN values:
 PriceHU         0
PMIHU           0
GAS             0
COAL            0
CO2             0
COALTOGAS       0
DOFFHU          0
DOFFSK          0
DOFFRO          0
WND             0
T2MALL          0
T2MALLMAX       0
T2MALLMIN       0
T2MALLAV        0
T2MALLNONLIN    0
T2MALLP         0
T2MALLPCO2      0
T2MALLPROR      0
T2MHUNORM       0
THUDEVNORM      0
T2MHUPSQ        0
T2MHUP          0
T2MHU           0
PRECHU          0
WS10MHU         0
ALLSKY1HU       0
T2MSKPSQ        0
T2MSKP          0
T2MSK           0
PRECSK          0
WS10MSK         0
ALLSKY1SK       0
T2MROPSQ        0
T2MROP          0
T2MRO           0
PRECRO          0
WS10MRO         0
ALLSKY1RO       0
UNAVNUCHU       0
UNAVGASHU       0
UNAVLIGNHU      0
UNAVTPPHU       0
UNAVNUCSK       0
UNAVGASSK       0
UNAVLIGNSK      0
UNAVTPPSK       0
UNAVHYDRSK      0
UNAVNUCRO       0
UNAVGASRO       0
UNAVLIGNRO      0
UNAVTPPRO       0
UNAVHYDRRO      0
UNAVNUCCZ       0
UNAVGASCZ       0
UNAVL

In [121]:
# Feature Engineering - Date Features
data['hour'] = data['Date'].dt.hour
data['day_of_week'] = data['Date'].dt.dayofweek
data['is_weekend'] = data['day_of_week'].isin([5, 6]).astype(int)
data['month'] = data['Date'].dt.month

In [122]:
check_NaN(data)

Number of NaN values:
 PriceHU         0
PMIHU           0
GAS             0
COAL            0
CO2             0
COALTOGAS       0
DOFFHU          0
DOFFSK          0
DOFFRO          0
WND             0
T2MALL          0
T2MALLMAX       0
T2MALLMIN       0
T2MALLAV        0
T2MALLNONLIN    0
T2MALLP         0
T2MALLPCO2      0
T2MALLPROR      0
T2MHUNORM       0
THUDEVNORM      0
T2MHUPSQ        0
T2MHUP          0
T2MHU           0
PRECHU          0
WS10MHU         0
ALLSKY1HU       0
T2MSKPSQ        0
T2MSKP          0
T2MSK           0
PRECSK          0
WS10MSK         0
ALLSKY1SK       0
T2MROPSQ        0
T2MROP          0
T2MRO           0
PRECRO          0
WS10MRO         0
ALLSKY1RO       0
UNAVNUCHU       0
UNAVGASHU       0
UNAVLIGNHU      0
UNAVTPPHU       0
UNAVNUCSK       0
UNAVGASSK       0
UNAVLIGNSK      0
UNAVTPPSK       0
UNAVHYDRSK      0
UNAVNUCRO       0
UNAVGASRO       0
UNAVLIGNRO      0
UNAVTPPRO       0
UNAVHYDRRO      0
UNAVNUCCZ       0
UNAVGASCZ       0
UNAVL

In [123]:
# Lagged Features
lagged_features = [1, 2, 3, 6, 12, 24]
for lag in lagged_features:
    data[f'lag_{lag}'] = data['PriceHU'].shift(lag)
print("Lagged features created.")

Lagged features created.


In [124]:
data = data.dropna()

In [125]:
check_NaN(data)

Number of NaN values:
 PriceHU         0
PMIHU           0
GAS             0
COAL            0
CO2             0
COALTOGAS       0
DOFFHU          0
DOFFSK          0
DOFFRO          0
WND             0
T2MALL          0
T2MALLMAX       0
T2MALLMIN       0
T2MALLAV        0
T2MALLNONLIN    0
T2MALLP         0
T2MALLPCO2      0
T2MALLPROR      0
T2MHUNORM       0
THUDEVNORM      0
T2MHUPSQ        0
T2MHUP          0
T2MHU           0
PRECHU          0
WS10MHU         0
ALLSKY1HU       0
T2MSKPSQ        0
T2MSKP          0
T2MSK           0
PRECSK          0
WS10MSK         0
ALLSKY1SK       0
T2MROPSQ        0
T2MROP          0
T2MRO           0
PRECRO          0
WS10MRO         0
ALLSKY1RO       0
UNAVNUCHU       0
UNAVGASHU       0
UNAVLIGNHU      0
UNAVTPPHU       0
UNAVNUCSK       0
UNAVGASSK       0
UNAVLIGNSK      0
UNAVTPPSK       0
UNAVHYDRSK      0
UNAVNUCRO       0
UNAVGASRO       0
UNAVLIGNRO      0
UNAVTPPRO       0
UNAVHYDRRO      0
UNAVNUCCZ       0
UNAVGASCZ       0
UNAVL

In [126]:
# Rolling Statistics (Keep only important ones)
rolling_windows = [3, 12, 24, 36, 48, 7 * 24]
for window in rolling_windows:
    data[f'rolling_mean_{window}h'] = data['PriceHU'].rolling(window=window).mean()
    data[f'rolling_std_{window}h'] = data['PriceHU'].rolling(window=window).std()


In [127]:
data = data.dropna()

In [128]:
check_NaN(data)

Number of NaN values:
 PriceHU              0
PMIHU                0
GAS                  0
COAL                 0
CO2                  0
COALTOGAS            0
DOFFHU               0
DOFFSK               0
DOFFRO               0
WND                  0
T2MALL               0
T2MALLMAX            0
T2MALLMIN            0
T2MALLAV             0
T2MALLNONLIN         0
T2MALLP              0
T2MALLPCO2           0
T2MALLPROR           0
T2MHUNORM            0
THUDEVNORM           0
T2MHUPSQ             0
T2MHUP               0
T2MHU                0
PRECHU               0
WS10MHU              0
ALLSKY1HU            0
T2MSKPSQ             0
T2MSKP               0
T2MSK                0
PRECSK               0
WS10MSK              0
ALLSKY1SK            0
T2MROPSQ             0
T2MROP               0
T2MRO                0
PRECRO               0
WS10MRO              0
ALLSKY1RO            0
UNAVNUCHU            0
UNAVGASHU            0
UNAVLIGNHU           0
UNAVTPPHU            0
UNAVNUCSK  

In [129]:
# Exponentially Weighted Moving Averages
ewm_windows = [3, 12, 24]
for span in ewm_windows:
    data[f'ewm_{span}h'] = data['PriceHU'].ewm(span=span).mean()

In [130]:
check_NaN(data)

Number of NaN values:
 PriceHU              0
PMIHU                0
GAS                  0
COAL                 0
CO2                  0
COALTOGAS            0
DOFFHU               0
DOFFSK               0
DOFFRO               0
WND                  0
T2MALL               0
T2MALLMAX            0
T2MALLMIN            0
T2MALLAV             0
T2MALLNONLIN         0
T2MALLP              0
T2MALLPCO2           0
T2MALLPROR           0
T2MHUNORM            0
THUDEVNORM           0
T2MHUPSQ             0
T2MHUP               0
T2MHU                0
PRECHU               0
WS10MHU              0
ALLSKY1HU            0
T2MSKPSQ             0
T2MSKP               0
T2MSK                0
PRECSK               0
WS10MSK              0
ALLSKY1SK            0
T2MROPSQ             0
T2MROP               0
T2MRO                0
PRECRO               0
WS10MRO              0
ALLSKY1RO            0
UNAVNUCHU            0
UNAVGASHU            0
UNAVLIGNHU           0
UNAVTPPHU            0
UNAVNUCSK  

In [131]:
# FFT Features (Keep only the top frequencies)
fft_vals = fft(data['PriceHU'].fillna(0).values)  # Handle NaN with fillna
fft_magnitude = np.abs(fft_vals)
N = 3  # Reduce the number of frequencies to keep to avoid feature explosion
top_frequencies = np.argsort(fft_magnitude)[-N:]

for i, freq_idx in enumerate(top_frequencies):
    data[f'fft_magnitude_{i+1}'] = fft_magnitude[freq_idx]
print("FFT features created.")

FFT features created.


In [132]:
check_NaN(data)

Number of NaN values:
 PriceHU              0
PMIHU                0
GAS                  0
COAL                 0
CO2                  0
COALTOGAS            0
DOFFHU               0
DOFFSK               0
DOFFRO               0
WND                  0
T2MALL               0
T2MALLMAX            0
T2MALLMIN            0
T2MALLAV             0
T2MALLNONLIN         0
T2MALLP              0
T2MALLPCO2           0
T2MALLPROR           0
T2MHUNORM            0
THUDEVNORM           0
T2MHUPSQ             0
T2MHUP               0
T2MHU                0
PRECHU               0
WS10MHU              0
ALLSKY1HU            0
T2MSKPSQ             0
T2MSKP               0
T2MSK                0
PRECSK               0
WS10MSK              0
ALLSKY1SK            0
T2MROPSQ             0
T2MROP               0
T2MRO                0
PRECRO               0
WS10MRO              0
ALLSKY1RO            0
UNAVNUCHU            0
UNAVGASHU            0
UNAVLIGNHU           0
UNAVTPPHU            0
UNAVNUCSK  

In [133]:
# Define feature matrix X and target y
X = data.drop(['PriceHU', 'Date'], axis=1)
y = data['PriceHU']


In [134]:
# Feature Importance using RandomForest and Mutual Information
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X, y)
rf_importances = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=False)

In [135]:
mi_scores = mutual_info_regression(X, y)
mi_scores = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)
print("Mutual information scores calculated.")
mi_scores

Mutual information scores calculated.


ewm_3h               2.038671
rolling_mean_3h      1.856320
lag_1                1.766299
lag_2                1.202585
ewm_12h              1.135868
lag_24               1.037633
ewm_24h              0.974792
lag_3                0.909942
rolling_mean_24h     0.903325
GAS                  0.899164
rolling_mean_12h     0.895602
COAL                 0.882766
rolling_mean_168h    0.853401
rolling_mean_36h     0.846748
rolling_mean_48h     0.833917
UNAVGASRO            0.787659
PMIHU                0.785217
COALTOGAS            0.756420
CO2                  0.735973
UNAVGASHU            0.691160
UNAVTPPBG            0.673824
rolling_std_168h     0.628602
UNAVTPPRO            0.620778
UNAVGASALL           0.614405
UNAVLIGNRO           0.607586
UNAVHYDRALL          0.598688
lag_12               0.585187
lag_6                0.578161
UNAVTPPHU            0.563356
UNAVTPPGR            0.556062
rolling_std_48h      0.548203
UNAVLIGNBG           0.546901
rolling_std_24h      0.537499
UNAVGASGR 

In [144]:
check_NaN(data)

Number of NaN values:
 PriceHU              0
PMIHU                0
GAS                  0
COAL                 0
CO2                  0
COALTOGAS            0
DOFFHU               0
DOFFSK               0
DOFFRO               0
WND                  0
T2MALL               0
T2MALLMAX            0
T2MALLMIN            0
T2MALLAV             0
T2MALLNONLIN         0
T2MALLP              0
T2MALLPCO2           0
T2MALLPROR           0
T2MHUNORM            0
THUDEVNORM           0
T2MHUPSQ             0
T2MHUP               0
T2MHU                0
PRECHU               0
WS10MHU              0
ALLSKY1HU            0
T2MSKPSQ             0
T2MSKP               0
T2MSK                0
PRECSK               0
WS10MSK              0
ALLSKY1SK            0
T2MROPSQ             0
T2MROP               0
T2MRO                0
PRECRO               0
WS10MRO              0
ALLSKY1RO            0
UNAVNUCHU            0
UNAVGASHU            0
UNAVLIGNHU           0
UNAVTPPHU            0
UNAVNUCSK  

In [145]:
# Debugging: Check for duplicate labels
print("RF Importance Index Length:", len(rf_importances.index))
print("MI Scores Index Length:", len(mi_scores.index))
print("Unique RF Importance Index Length:", len(set(rf_importances.index)))
print("Unique MI Scores Index Length:", len(set(mi_scores.index)))

RF Importance Index Length: 129
MI Scores Index Length: 129
Unique RF Importance Index Length: 129
Unique MI Scores Index Length: 129


In [146]:
# Normalize and combine the scores
feature_scores = pd.DataFrame({'RF_Score': rf_importances, 'MI_Score': mi_scores})
feature_scores['CombinedScore'] = (feature_scores['RF_Score'] + feature_scores['MI_Score']).rank()
print("Feature scores combined.")

Feature scores combined.


In [147]:
# Select top features - Reduce to top 10-15
top_features = feature_scores.sort_values('CombinedScore', ascending=False).head(10).index.tolist()

In [148]:
top_features

['ewm_3h',
 'rolling_mean_3h',
 'lag_1',
 'lag_2',
 'ewm_12h',
 'lag_24',
 'ewm_24h',
 'lag_3',
 'rolling_mean_24h',
 'GAS']

In [149]:
# Generate Polynomial Features (Degree 2) only for top selected features
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
poly_features = poly.fit_transform(data[top_features])
poly_feature_names = poly.get_feature_names_out(top_features)
poly_df = pd.DataFrame(poly_features, columns=poly_feature_names)

In [150]:
check_NaN(data)

Number of NaN values:
 PriceHU              0
PMIHU                0
GAS                  0
COAL                 0
CO2                  0
COALTOGAS            0
DOFFHU               0
DOFFSK               0
DOFFRO               0
WND                  0
T2MALL               0
T2MALLMAX            0
T2MALLMIN            0
T2MALLAV             0
T2MALLNONLIN         0
T2MALLP              0
T2MALLPCO2           0
T2MALLPROR           0
T2MHUNORM            0
THUDEVNORM           0
T2MHUPSQ             0
T2MHUP               0
T2MHU                0
PRECHU               0
WS10MHU              0
ALLSKY1HU            0
T2MSKPSQ             0
T2MSKP               0
T2MSK                0
PRECSK               0
WS10MSK              0
ALLSKY1SK            0
T2MROPSQ             0
T2MROP               0
T2MRO                0
PRECRO               0
WS10MRO              0
ALLSKY1RO            0
UNAVNUCHU            0
UNAVGASHU            0
UNAVLIGNHU           0
UNAVTPPHU            0
UNAVNUCSK  

In [151]:
check_NaN(poly_df)

Number of NaN values:
 ewm_3h                              0
rolling_mean_3h                     0
lag_1                               0
lag_2                               0
ewm_12h                             0
lag_24                              0
ewm_24h                             0
lag_3                               0
rolling_mean_24h                    0
GAS                                 0
ewm_3h^2                            0
ewm_3h rolling_mean_3h              0
ewm_3h lag_1                        0
ewm_3h lag_2                        0
ewm_3h ewm_12h                      0
ewm_3h lag_24                       0
ewm_3h ewm_24h                      0
ewm_3h lag_3                        0
ewm_3h rolling_mean_24h             0
ewm_3h GAS                          0
rolling_mean_3h^2                   0
rolling_mean_3h lag_1               0
rolling_mean_3h lag_2               0
rolling_mean_3h ewm_12h             0
rolling_mean_3h lag_24              0
rolling_mean_3h ewm_24h    

In [152]:
# Ensure that the indices of poly_df match those of the original data
poly_df.index = data.index

In [153]:
# Identify columns in poly_df that do not exist in data
new_columns = [col for col in poly_df.columns if col not in data.columns]
new_columns

['ewm_3h^2',
 'ewm_3h rolling_mean_3h',
 'ewm_3h lag_1',
 'ewm_3h lag_2',
 'ewm_3h ewm_12h',
 'ewm_3h lag_24',
 'ewm_3h ewm_24h',
 'ewm_3h lag_3',
 'ewm_3h rolling_mean_24h',
 'ewm_3h GAS',
 'rolling_mean_3h^2',
 'rolling_mean_3h lag_1',
 'rolling_mean_3h lag_2',
 'rolling_mean_3h ewm_12h',
 'rolling_mean_3h lag_24',
 'rolling_mean_3h ewm_24h',
 'rolling_mean_3h lag_3',
 'rolling_mean_3h rolling_mean_24h',
 'rolling_mean_3h GAS',
 'lag_1^2',
 'lag_1 lag_2',
 'lag_1 ewm_12h',
 'lag_1 lag_24',
 'lag_1 ewm_24h',
 'lag_1 lag_3',
 'lag_1 rolling_mean_24h',
 'lag_1 GAS',
 'lag_2^2',
 'lag_2 ewm_12h',
 'lag_2 lag_24',
 'lag_2 ewm_24h',
 'lag_2 lag_3',
 'lag_2 rolling_mean_24h',
 'lag_2 GAS',
 'ewm_12h^2',
 'ewm_12h lag_24',
 'ewm_12h ewm_24h',
 'ewm_12h lag_3',
 'ewm_12h rolling_mean_24h',
 'ewm_12h GAS',
 'lag_24^2',
 'lag_24 ewm_24h',
 'lag_24 lag_3',
 'lag_24 rolling_mean_24h',
 'lag_24 GAS',
 'ewm_24h^2',
 'ewm_24h lag_3',
 'ewm_24h rolling_mean_24h',
 'ewm_24h GAS',
 'lag_3^2',
 'lag_3 r

In [154]:
# data_backup = data.copy()

In [155]:
# Only add the unique columns from poly_df to data
data = pd.concat([data, poly_df[new_columns]], axis=1)

In [156]:
check_NaN(data)

Number of NaN values:
 PriceHU                             0
PMIHU                               0
GAS                                 0
COAL                                0
CO2                                 0
COALTOGAS                           0
DOFFHU                              0
DOFFSK                              0
DOFFRO                              0
WND                                 0
T2MALL                              0
T2MALLMAX                           0
T2MALLMIN                           0
T2MALLAV                            0
T2MALLNONLIN                        0
T2MALLP                             0
T2MALLPCO2                          0
T2MALLPROR                          0
T2MHUNORM                           0
THUDEVNORM                          0
T2MHUPSQ                            0
T2MHUP                              0
T2MHU                               0
PRECHU                              0
WS10MHU                             0
ALLSKY1HU                  

In [157]:
# Remove Multicollinear Features
corr_matrix = data.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
highly_correlated = [column for column in upper.columns if any(upper[column] > 0.9)]
selected_features = list(set(data.columns) - set(highly_correlated) - {'PriceHU', 'Date'})

In [158]:
# Applying PCA to find the best number of components
pca_full = PCA().fit(data[selected_features])
explained_variances = np.cumsum(pca_full.explained_variance_ratio_)

# Find the number of components that explain at least 95% of the variance
best_n_components = np.argmax(explained_variances >= 0.95) + 1
print(f"Number of PCA components explaining at least 95% of variance: {best_n_components}")

Number of PCA components explaining at least 95% of variance: 5


In [159]:
# Applying PCA to further reduce the dimensionality
pca = PCA(n_components=5)  # Adjust the number of components based on the explained variance or domain needs
pca_components = pca.fit_transform(data[selected_features])
print(f"PCA applied to reduce dimensionality to {pca.n_components} components.")

PCA applied to reduce dimensionality to 5 components.


In [160]:
# Create a DataFrame for PCA components
pca_columns = [f'PCA_{i+1}' for i in range(pca_components.shape[1])]
pca_df = pd.DataFrame(pca_components, columns=pca_columns)
print("PCA components DataFrame created.")

PCA components DataFrame created.


In [161]:
# Prepare Data for Prophet Regressors
data_for_prophet = pd.concat([data[['Date', 'PriceHU']], pca_df], axis=1)
data_for_prophet = data_for_prophet.rename(columns={'Date': 'ds', 'PriceHU': 'y'})
print("Data prepared for Prophet with PCA components as regressors.")

Data prepared for Prophet with PCA components as regressors.


In [162]:
# Print Final Selected Features
print("Final PCA Components for Prophet Regressors:", pca_columns)

Final PCA Components for Prophet Regressors: ['PCA_1', 'PCA_2', 'PCA_3', 'PCA_4', 'PCA_5']


In [163]:
len(pca_columns)

5

In [164]:
data_for_prophet.head()

,ds,y,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5
8975,2017-01-08 23:00:00,57.69,-19730.418658,20523.828086,-19056.228537,13802.085761,-3089.571378
8976,2017-01-09 00:00:00,46.37,-20764.377981,20095.408964,-18993.955976,13873.859300,-3012.524664
8977,2017-01-09 01:00:00,40.36,-21361.097481,19527.503809,-18936.980887,13819.132053,-3016.169447
8978,2017-01-09 02:00:00,38.38,-20670.507585,19947.930032,-18920.254799,13730.454836,-3058.845597
8979,2017-01-09 03:00:00,38.55,-22632.668940,19326.838555,-18865.672695,14040.024173,-2897.408393


In [165]:
# Export Data for Prophet
data_for_prophet.to_csv('prepared_data_for_prophet.csv', index=False)
print("Data exported to 'prepared_data_for_prophet.csv'.")

Data exported to 'prepared_data_for_prophet.csv'.
